In [27]:
print('hello, world!')

hello, world!


In [28]:
import os
from data import SQLData
from datasets import Dataset
from dotenv import load_dotenv
import datasets

load_dotenv("../../.env")

True

In [29]:
HUGGING_FACE_WRITE_TOKEN = os.getenv("HUGGING_FACE_WRITE_TOKEN")

hf_spider_dev = 'alagaesia/spider_dev'
hf_spider_dev_preds = 'alagaesia/spider_dev_preds'

In [30]:
sqp = SQLData.from_dataset(hf_spider_dev)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [31]:
#sqp.load_data(hf_spider_dev_preds)

In [32]:
sqp.data[hf_spider_dev]['train'][0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'question': 'How many singers do we have?',
 'create_w_keys': 'CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") ); ',
 'create_wo_keys': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity

In [33]:

def dep_request_creation(
    strings: list[str],
    create_statement: str,
    question: str,
    create_statement_index: int,
    question_index: int,
) -> str: 
    
    modified_strings = strings.copy()
    
    modified_strings.insert(create_statement_index, create_statement)
    modified_strings.insert(question_index, question)

    return ' '.join(modified_strings)

def dep_dataset_request_creation(
    dataset: Dataset,
    strings: list[str],
    request_column: str,
    create_statement_column: str,
    question_column: str,
    create_statement_index: int,
    question_index: int,
) -> str: 
    
    create_statement = dataset[create_statement_column]
    question = dataset[question_column]

    request_string = dep_request_creation(
        strings,
        create_statement,
        question,
        create_statement_index,
        question_index,
    )

    return {request_column: request_string}

request_column = 'zero_shot_request_complex'
create_statement_column = 'create_w_keys'
question_column = 'question'

strings = ['Given the following SQL schema: ', 'Provide a SQL query that answers the following question: ', 'Only respond with the SQL query.']

strings = [
    '### Complete sqlite SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query.\n',
    '### Sqlite SQL tables, with their properties:\n',
    '#\n',
    '# ', # create statement goes here (index 4)
    '\n', 
    '#\n',
    '### ', # question goes here (index 8) - must account for prior insertion
    '\n', 
    'SQL QUERY:',
]

create_statement_index = 4
question_index = 8

sqp.data[hf_spider_dev]['train'] = sqp.data[hf_spider_dev]['train'].map(dep_dataset_request_creation, fn_kwargs = {'strings': strings, 'request_column': request_column, 'create_statement_column': create_statement_column, 'question_column': question_column, 'create_statement_index': create_statement_index, 'question_index': question_index}, load_from_cache_file=False)

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [35]:
sqp.data[hf_spider_dev]['train'][0]['zero_shot_request_complex']

'### Complete sqlite SQL query only and with no explanation, and do not select extra columns that are not explicitly requested in the query.\n ### Sqlite SQL tables, with their properties:\n #\n #  CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") );  \n #\n ###  How many singers 

In [36]:
sqp.data[hf_spider_dev]['train']

Dataset({
    features: ['db_id', 'query', 'question', 'create_w_keys', 'create_wo_keys', 'difficulty', 'zero_shot_request_complex'],
    num_rows: 1034
})

In [37]:
def request_creation(
    create_statement: str, 
    question: str,
):
    return f"Provide the SQL query that answers the QUESTION, with no explanation or special characters. CONTEXT: Relevant SQL tables, with their properties: {create_statement} QUESTION: {question} "

def dataset_request_creation(
    dataset: Dataset,
    request_column: str,
    create_statement_column: str,
    question_column: str,
):
    
    create_statement = dataset[create_statement_column]
    question = dataset[question_column]

    request_string = request_creation(
        create_statement,
        question,
    )

    return {request_column: request_string}

In [38]:
request_column = 'zero_shot_request'
create_statement_column = 'create_w_keys'
question_column = 'question'

In [39]:
sqp.data[hf_spider_dev]['train'] = sqp.data[hf_spider_dev]['train'].map(dataset_request_creation, fn_kwargs = {'request_column': request_column, 'create_statement_column': create_statement_column, 'question_column': question_column}, load_from_cache_file=False)

Map:   0%|          | 0/1034 [00:00<?, ? examples/s]

In [40]:
sqp.data[hf_spider_dev]['train'][0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'question': 'How many singers do we have?',
 'create_w_keys': 'CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") ); ',
 'create_wo_keys': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity

In [12]:
print(sqp.data[hf_spider_dev]['train'][0]['zero_shot_request'])

Provide the SQL query that answers the QUESTION, with no explanation or special characters. CONTEXT: Relevant SQL tables, with their properties: CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") );  QUESTION: How many singers do we have? 


In [ ]:
'''
sqp.hub_upload(
    repo_id=hf_spider_dev_preds,
    token=HUGGING_FACE_WRITE_TOKEN,
    dataset=sqp.data[hf_spider_dev],
)
'''

In [13]:
sqp.data[hf_spider_dev]['train'][0]

{'db_id': 'concert_singer',
 'query': 'SELECT count(*) FROM singer',
 'question': 'How many singers do we have?',
 'create_w_keys': 'CREATE TABLE "stadium" ( "Stadium_ID" int, "Location" text, "Name" text, "Capacity" int, "Highest" int, "Lowest" int, "Average" int, PRIMARY KEY ("Stadium_ID") ); CREATE TABLE "singer" ( "Singer_ID" int, "Name" text, "Country" text, "Song_Name" text, "Song_release_year" text, "Age" int, "Is_male" bool, PRIMARY KEY ("Singer_ID") ); CREATE TABLE "concert" ( "concert_ID" int, "concert_Name" text, "Theme" text, "Stadium_ID" text, "Year" text, PRIMARY KEY ("concert_ID"), FOREIGN KEY ("Stadium_ID") REFERENCES "stadium"("Stadium_ID") ); CREATE TABLE "singer_in_concert" ( "concert_ID" int, "Singer_ID" text, PRIMARY KEY ("concert_ID","Singer_ID"), FOREIGN KEY ("concert_ID") REFERENCES "concert"("concert_ID"), FOREIGN KEY ("Singer_ID") REFERENCES "singer"("Singer_ID") ); ',
 'create_wo_keys': 'CREATE TABLE stadium (Stadium_ID INT, Location TEXT, Name TEXT, Capacity

In [14]:
goal = 200
goal_per_each = 3
db_ids = {}
subset = []

db_queries = {}

sqp.data[hf_spider_dev]['train'][0]

for i in range(len(sqp.data[hf_spider_dev]['train'])):

    db_id = sqp.data[hf_spider_dev]['train'][i]['db_id']
    difficulty = sqp.data[hf_spider_dev]['train'][i]['difficulty']
    query = sqp.data[hf_spider_dev]['train'][i]['query']

    if db_id not in db_ids:
        db_ids[db_id] = {}
        db_ids[db_id][difficulty] = 1

        db_queries[db_id] = {}
        db_queries[db_id][difficulty] = [query]

        subset.append(sqp.data[hf_spider_dev]['train'][i])
    else:
        if difficulty not in db_ids[db_id]:
            db_ids[db_id][difficulty] = 1

            db_queries[db_id][difficulty] = [query]

            subset.append(sqp.data[hf_spider_dev]['train'][i])
        else:
            if db_ids[db_id][difficulty] < goal_per_each and query not in db_queries[db_id][difficulty]:
                db_ids[db_id][difficulty] += 1

                db_queries[db_id][difficulty].append(query)

                subset.append(sqp.data[hf_spider_dev]['train'][i])
            else: 
                continue

In [16]:
curated_dataset = Dataset.from_dict({k: [item[k] for item in subset] for k in subset[0]})


In [17]:
curated_dataset['query']

['SELECT count(*) FROM singer',
 'SELECT name ,  country ,  age FROM singer ORDER BY age DESC',
 "SELECT avg(age) ,  min(age) ,  max(age) FROM singer WHERE country  =  'France'",
 'SELECT song_name ,  song_release_year FROM singer ORDER BY age LIMIT 1',
 'SELECT DISTINCT country FROM singer WHERE age  >  20',
 'SELECT song_name FROM singer WHERE age  >  (SELECT avg(age) FROM singer)',
 'SELECT T2.name ,  T2.capacity FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium_id  =  T2.stadium_id WHERE T1.year  >=  2014 GROUP BY T2.stadium_id ORDER BY count(*) DESC LIMIT 1',
 'select t2.name ,  t2.capacity from concert as t1 join stadium as t2 on t1.stadium_id  =  t2.stadium_id where t1.year  >  2013 group by t2.stadium_id order by count(*) desc limit 1',
 'SELECT YEAR FROM concert GROUP BY YEAR ORDER BY count(*) DESC LIMIT 1',
 'SELECT name FROM stadium WHERE stadium_id NOT IN (SELECT stadium_id FROM concert)',
 'SELECT T2.name ,  T2.location FROM concert AS T1 JOIN stadium AS T2 ON T1.stadium

In [23]:
curated_dataset.push_to_hub('alagaesia/spider_dev_subset_preds', token=HUGGING_FACE_WRITE_TOKEN)

Pushing dataset shards to the dataset hub:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

In [24]:
curated_dataset.save_to_disk('spider_dev_subset_preds')

Saving the dataset (0/1 shards):   0%|          | 0/209 [00:00<?, ? examples/s]

In [ ]:
total = 0 
for key in db_ids.keys():
    print(key)
    tot = 0 
    for val in db_ids[key].keys():
        tot += db_ids[key][val]
    print(tot)
    #print(db_ids[key])
    total += tot

print(total)

In [ ]:
sqp.data[hf_spider_dev]